In [4]:
# ====================
# Feature Extraction
# ====================
import numpy as np
import librosa
import joblib
from tensorflow import keras

import librosa
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        delta_mfcc = librosa.feature.delta(mfccs)
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)
        mfcc_features = np.hstack((
            np.mean(mfccs, axis=1),
            np.mean(delta_mfcc, axis=1),
            np.mean(delta2_mfcc, axis=1)
        ))

        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_features = np.mean(chroma, axis=1)

        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_features = np.mean(mel, axis=1)

        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        contrast_features = np.mean(spectral_contrast, axis=1)

        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        tonnetz_features = np.mean(tonnetz, axis=1)

        combined = np.hstack((
            mfcc_features, chroma_features, mel_features, contrast_features, tonnetz_features
        ))

        return combined
    except Exception as e:
        print(f"Feature extraction error: {e}")
        return np.zeros(192)  # fallback feature shape

In [13]:
# ====================
# Prediction Function
# ====================
import joblib

def predict_audio(file_path):
    try:
        # Load scaler and label encoder
        scaler = joblib.load('scaler.pkl')
        label_encoder = joblib.load('label_encoder.pkl')

        # Extract and scale features
        features = extract_features(file_path).reshape(1, -1)
        features_scaled = scaler.transform(features)

        # ---------- DNN Prediction ----------
        dnn_model = keras.models.load_model('DNN/DNN-Model.keras')
        dnn_pred = dnn_model.predict(features_scaled)
        dnn_label = label_encoder.inverse_transform([np.argmax(dnn_pred)])[0]

        # ---------- CNN Prediction ----------
        cnn_input = features_scaled.reshape(1, features_scaled.shape[1], 1, 1)
        cnn_model = keras.models.load_model('CNN/cnn-model.keras')
        cnn_pred = cnn_model.predict(cnn_input)
        cnn_label = label_encoder.inverse_transform([np.argmax(cnn_pred)])[0]

        return {
            "DNN Prediction": dnn_label,
            "CNN Prediction": cnn_label
        }

    except Exception as e:
        print(f"Prediction error: {e}")
        return None

In [12]:
# ====================
# Example Prediction
# ====================
file_path = "spoof/tts-0162000418.wav"
results = predict_audio(file_path)
if results:
    print("DNN Prediction:", results["DNN Prediction"])
    print("CNN Prediction:", results["CNN Prediction"])


c:\Users\PAVAN KUMAR R\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\PAVAN KUMAR R\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
DNN Prediction: 0
CNN Prediction: 0
